In [1]:
import sys
import os
import py_entitymatching as em
print('magellan version:' + em.__version__)
import re
import csv
import pandas as pd

from cleaner import Cleaner
from constants import csv_headers

magellan version:0.1.0


In [2]:
working_dir = os.path.dirname(os.getcwd())
path_to_csv_dir = working_dir + os.sep + 'csv_files'+ os.sep

AOM = em.read_csv_metadata(path_to_csv_dir + '_aom_cleaned.csv', key = csv_headers.AOM_INDEX)
WHED = em.read_csv_metadata(path_to_csv_dir + '_whed_cleaned.csv', key = csv_headers.WHED_INDEX)

#sample_AOM = em.read_csv_metadata(path_to_csv_dir + 'sample_AOM.csv', key = csv_headers.AOM_INDEX)
#sample_WHED = em.read_csv_metadata(path_to_csv_dir + 'sample_WHED.csv', key = csv_headers.WHED_INDEX)

C = em.load_table(path_to_csv_dir + 'matching_pairs_table_overlap3_emailserver.pkl')
em.set_ltable(C, WHED)
em.set_rtable(C, AOM)

print(C.size)
print(em.show_properties(C))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


126516
id: 4647164728
key: _id
fk_ltable: ltable_a_id
fk_rtable: rtable_person_id
ltable(obj.id): 4647164504
rtable(obj.id): 4542100872
None


In [153]:
G_all = []
num_set = 2
for i in range(num_set):
    S = em.sample_table(C, 30)
    G = em.label_table(S, 'gold_label')
    em.to_csv_metadata(G,path_to_csv_dir + 'G_new' + str(i) + '.csv')   
    G_all.append(G)

count_all = []
for i in range(num_set):
    tmp = G_all[i]['gold_label']
    count_all.append((sum(tmp)))
ratio = [i/30 for i in count_all]
ratio

[0.40000000000000002, 0.29999999999999999]

In [3]:
training_size = 500
S = em.sample_table(C, training_size)
G = em.label_table(S, 'gold_label')
em.to_csv_metadata(G,path_to_csv_dir + 'golden_data.csv')   

Column name (gold_label) is not present in dataframe


True

In [ ]:

G1 = em.read_csv_metadata(path_to_csv_dir + 'G1.csv',key='_id',
                          ltable=WHED, rtable=AOM, 
                          fk_ltable='ltable_a_id', fk_rtable='rtable_person_id') 
G2 = em.read_csv_metadata(path_to_csv_dir + 'G2.csv',key='_id',
                          ltable=WHED, rtable=AOM, 
                          fk_ltable='ltable_a_id', fk_rtable='rtable_person_id') 
# G1.head()

In [88]:
#ft_table = em.create_feature_table()
ft_table = em.getFeature
def is_same_city(ltuple, rtuple):
    # dealing with missing data, if either table is missing city, return 0
    if pd.isnull(ltuple['a_city']) or pd.isnull(rtuple['a_city']):
        return 0
    # return 1 if is same city, 0 otherwise
    if ltuple['a_city'] == rtuple['a_city']:
        return 1
    else:
        return 0

def is_same_prov(ltuple, rtuple):
    # dealing with missing data, if either table is missing city, return 0
    if pd.isnull(ltuple['a_prov']) or pd.isnull(rtuple['a_prov']):
        return 0
    # return 1 if is same city, 0 otherwise
    if ltuple['a_prov'] == rtuple['a_prov']:
        return 1
    else:
        return 0    

def is_same_country(ltuple, rtuple):
    # dealing with missing data, if either table is missing city, return 0
    if pd.isnull(ltuple['a_country']) or pd.isnull(rtuple['a_country']):
        return 0
    # return 1 if is same city, 0 otherwise
    if ltuple['a_country'] == rtuple['a_country']:
        return 1
    else:
        return 0      

def is_same_server(ltuple, rtuple):
    # Attention, the input argument is not symmetric,
    # dealing with missing data, if either table is missing city, return 0
    if pd.isnull(ltuple['a_web']) or pd.isnull(rtuple['a_email_server']):
        return 0
    # return 1 if is same city, 0 otherwise
    if ltuple['a_web'] in rtuple['a_email_server'].split('.'):
        return 1
    else:
        return 0       
status = em.add_blackbox_feature(ft_table, 'is_same_city', is_same_city)
status = em.add_blackbox_feature(ft_table, 'is_same_prov', is_same_prov)
status = em.add_blackbox_feature(ft_table, 'is_same_country', is_same_country)
status = em.add_blackbox_feature(ft_table, 'is_same_server', is_same_server)

attrs_from_table = ['ltable_a_name','ltable_a_city','ltable_a_prov','ltable_a_country','ltable_a_web',
                    'rtable_a_name','rtable_a_city','rtable_a_prov','rtable_a_country','rtable_a_email_server']


In [128]:
H = em.extract_feature_vecs(G1, 
                            feature_table=ft_table, 
                            attrs_before = attrs_from_table,
                            attrs_after='gold_label',
                            show_progress=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [94]:
rf = em.RFMatcher()
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_a_id', 'rtable_person_id', 'gold_label'])
attrs_to_be_excluded.extend(attrs_from_table)
rf.fit(table=H, exclude_attrs=attrs_to_be_excluded, target_attr='gold_label')

In [95]:
L = em.extract_feature_vecs(G2, 
                            feature_table=ft_table, 
                            attrs_before = attrs_from_table,
                            attrs_after='gold_label',
                            show_progress=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [96]:
# Predict the matches
predictions = rf.predict(table=L, exclude_attrs=attrs_to_be_excluded,                          
              append=True, target_attr='predicted', inplace=False)



In [98]:
predictions.head(n = 30)



,_id,ltable_a_id,rtable_person_id,ltable_a_name,ltable_a_city,ltable_a_prov,ltable_a_country,ltable_a_web,rtable_a_name,rtable_a_city,rtable_a_prov,rtable_a_country,rtable_a_email_server,is_same_city,is_same_prov,is_same_country,is_same_server,gold_label,predicted
0,13,10,22207,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,hanken,0,0,1,0,1,0
1,17,15,58875,aarhus university,aarhus,C,denmark,http://www.au.dk/en,aarhus university,aarhus,NaN,denmark,au,1,0,1,0,1,1
2,41,917,58552,american university beirut,beirut,NaN,lebanon,http://www.aub.edu.lb,american university beirut,beirut,NaN,lebanon,aub,1,0,1,0,1,1
3,96,1997,6611,carleton university,ottawa,Ontario,canada,http://www.carleton.ca/,carleton university,ottawa,on,canada,carleton,1,0,1,0,1,1
4,148,3809,46745,lower silesian university entrepreneurship technology polkowice,polkowice,dolnoslaskie,poland,http://www.dwspit.pl,silesian university technology,zabrze,NaN,poland,pl,0,0,1,0,0,0
5,166,4508,39215,erasmus university rotterdam,rotterdam,NaN,netherlands,http://www.eur.nl,erasmus university rotterdam,rotterdam,NaN,netherlands,rsm,1,0,1,0,1,1
6,200,4849,56644,european distance education university applied sciences hamburg,hamburg,NaN,germany,http://www.euro-fh.com,university hamburg,hamburg,hamburg,germany,uni-hamburg,1,0,1,0,0,1
7,241,6429,58451,friedrich alexander university erlangen nuremberg,erlangen,Bavaria,germany,http://www.fau.de,friedrich alexander university erlangen nuremberg,nuremberg,NaN,germany,web,0,0,1,0,1,0
8,525,10528,59371,korea national sport university,seoul,NaN,south korea,http://www.knsu.ac.kr/web/kor/home,korea university,seoul,NaN,south korea,korea,1,0,1,0,0,1
9,533,10530,24053,korea national university transportation,chungju si,Chungbuk-do,south korea,http://www.ut.ac.kr/,korea university,seoul,NaN,south korea,korea,0,0,1,0,0,0


False
